In [1]:
import os
import cv2
import numpy as np
import pandas as pd

In [14]:
for study in os.listdir("Studies/"):
    if not study.endswith("-0"):
        for orientation in os.listdir("Studies/"+study+"/"):
            for folder in os.listdir("Studies/"+study+"/"+orientation+"/"):
                if folder == "anno":
                    df = pd.read_csv("Studies/"+study+"/"+orientation+"/"+folder+"/gTruth.csv").dropna().reset_index().drop(columns="index")
                    df["PixelLabelData"] = df["PixelLabelData"].map(lambda path: "/".join(["Studies"]+path.split("/")[-5:]))
                    df["OriginalData"] = ""
                    try:
                        os.mkdir("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/")
                        vidcap = cv2.VideoCapture("Studies/"+study+"/"+orientation+"/clip/"+orientation+".mp4")
                        for i in range(len(df)):
                            name = df.at[i, 'PixelLabelData'].split("/")[-1]
                            time = round(float(df.at[i, 'Time'].split(" ")[0])*1000)
                            vidcap.set(cv2.CAP_PROP_POS_MSEC, time)
                            success,image = vidcap.read()
                            if success:
                                cv2.imwrite("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name, image)
                                df.at[i, "OriginalData"] = "Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name
                        df = df.replace('', np.nan).dropna().reset_index().drop(columns="index")
                        df.to_csv("Studies/"+study+"/"+orientation+"/"+"data.csv", index=False)
                    except:
                        print("folder done")
    else:
        for orientation in os.listdir("Studies/"+study+"/"):
            for folder in os.listdir("Studies/"+study+"/"+orientation+"/"):
                if folder == "anno":
                    df = pd.read_csv("Studies/"+study+"/"+orientation+"/"+folder+"/gTruth.csv").dropna().reset_index().drop(columns="index")
                    df["PixelLabelData"] = df["PixelLabelData"].map(lambda path: "/".join(["Studies"]+path.split("\\")[-5:]))
                    df["OriginalData"] = ""
                    try:
                        os.mkdir("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/")
                        vidcap = cv2.VideoCapture("Studies/"+study+"/"+orientation+"/clip/"+orientation+".mp4")
                        for i in range(len(df)):
                            name = df.at[i, 'PixelLabelData'].split("/")[-1]
                            time = round(float(df.at[i, 'Time'].split(" ")[0])*1000)
                            vidcap.set(cv2.CAP_PROP_POS_MSEC, time)
                            success,image = vidcap.read()
                            if success:
                                cv2.imwrite("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name, image)
                                df.at[i, "OriginalData"] = "Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name
                        df = df.replace('', np.nan).dropna().reset_index().drop(columns="index")
                        df.to_csv("Studies/"+study+"/"+orientation+"/"+"data.csv", index=False)
                    except:
                        print("folder done")

folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done
folder done


In [15]:
finaldf = []
for study in os.listdir("Studies/"):
    for orientation in os.listdir("Studies/"+study+"/"):
        for fsobject in os.listdir("Studies/"+study+"/"+orientation+"/"):
            if fsobject == "data.csv":
                df = pd.read_csv("Studies/"+study+"/"+orientation+"/"+fsobject)
                finaldf.append(df)
finaldf = pd.concat(finaldf, ignore_index=True)
finaldf.to_csv("dataset.csv", index=False)

In [16]:
dataset = pd.read_csv("dataset.csv")
for i in range(len(dataset)):
        image = cv2.imread(dataset.at[i, 'OriginalData'], cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(dataset.at[i, 'PixelLabelData'], cv2.IMREAD_GRAYSCALE)
        if (sum(sum(mask)) > 0):
            (thresh, im_bw) = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
            cv2.addWeighted(im_bw, 0.7, image, 1, 0, image)
            filename = dataset.at[i, 'OriginalData'].split("/")[-1].split(".")[0]
            path = "/".join(dataset.at[i, 'OriginalData'].split("/")[:3])+"/visualizations"
            try:
                os.mkdir(path)
                cv2.imwrite(path+"/"+filename+"_masked.png", image)
            except:
                cv2.imwrite(path+"/"+filename+"_masked.png", image)

In [17]:
dataset = pd.read_csv("dataset.csv")
train_split = 0.8
try:
    os.mkdir("data/")    
    os.mkdir("data/train/")
    os.mkdir("data/train/images")
    os.mkdir("data/train/masks")
    os.mkdir("data/val/")
    os.mkdir("data/val/images")
    os.mkdir("data/val/masks")
    for i in range(len(dataset)):
        image = cv2.imread(dataset.at[i, 'OriginalData'])
        mask = cv2.imread(dataset.at[i, 'PixelLabelData'], cv2.IMREAD_GRAYSCALE)
        if (sum(sum(mask)) > 0):
            p = np.random.random()
            if (p <= train_split):
                cv2.imwrite("data/train/images/"+str(i)+".png", image)
                cv2.imwrite("data/train/masks/"+str(i)+".png", mask)
            else:
                cv2.imwrite("data/val/images/"+str(i)+".png", image)
                cv2.imwrite("data/val/masks/"+str(i)+".png", mask)
except:
    for i in range(len(dataset)):
        image = cv2.imread(dataset.at[i, 'OriginalData'])
        mask = cv2.imread(dataset.at[i, 'PixelLabelData'], cv2.IMREAD_GRAYSCALE)
        if (sum(sum(mask)) > 0):
            p = np.random.random()
            if (p <= train_split):
                cv2.imwrite("data/train/images/"+str(i)+".png", image)
                cv2.imwrite("data/train/masks/"+str(i)+".png", mask)
            else:
                cv2.imwrite("data/val/images/"+str(i)+".png", image)
                cv2.imwrite("data/val/masks/"+str(i)+".png", mask)